In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

In [ ]:
df_keywords_titles = pd.read_json('/mnt/hdd01/patentsview/Similarity Search - CPC Classification and Claims/cpc_yake_keywords_list_noun_chunks_embeddings.json')

# Convert columns 'keyword_patentsberta_embedding', 'keyword_climatebert_embedding', 'keyword_bertforpatents_embedding' to numpy arrays
df_keywords_titles['keyword_yake_patentsberta_embedding'] = df_keywords_titles['keyword_yake_patentsberta_embedding'].progress_apply(lambda x: np.array(x, dtype=np.float32))
df_keywords_titles['keyword_yake_climatebert_embedding'] = df_keywords_titles['keyword_yake_climatebert_embedding'].progress_apply(lambda x: np.array(x, dtype=np.float32))
df_keywords_titles['keyword_yake_bertforpatents_embedding'] = df_keywords_titles['keyword_yake_bertforpatents_embedding'].progress_apply(lambda x: np.array(x, dtype=np.float32))

In [ ]:
df_keywords_claims = pd.read_json('/mnt/hdd01/patentsview/Similarity Search - CPC Classification and Claims/df_keywords_list_agg_uspto_epo_rel_embeddings_noun_chunks.json')

# Convert columns 'keyword_yake_patentsberta_embedding', 'keyword_yake_climatebert_embedding', 'keyword_yake_bertforpatents_embedding' to numpy arrays
df_keywords_claims['keyword_yake_patentsberta_embedding'] = df_keywords_claims['keyword_yake_patentsberta_embedding'].progress_apply(lambda x: np.array(x, dtype=np.float32))
df_keywords_claims['keyword_yake_climatebert_embedding'] = df_keywords_claims['keyword_yake_climatebert_embedding'].progress_apply(lambda x: np.array(x, dtype=np.float32))
df_keywords_claims['keyword_yake_bertforpatents_embedding'] = df_keywords_claims['keyword_yake_bertforpatents_embedding'].progress_apply(lambda x: np.array(x, dtype=np.float32))

# Testing

In [ ]:
# Print out all keyword_yake_lemma for cpc_class_symbol = Y02W
print(df_keywords_titles[df_keywords_titles['cpc_class_symbol'].progress_apply(lambda x: 'Y02W10/33' in x)][['keyword_yake_lemma', 'cpc_class_symbol']])

In [ ]:
# Set search test to row with keyword = solar cells
df_search_test_titles = df_keywords_titles[df_keywords_titles['keyword_yake_lemma'] == 'wind energy']

In [ ]:
# Reset index of df_search_test_titles
df_search_test_titles.reset_index(drop=True, inplace=True)

In [ ]:
df_search_test_titles

# Similarity Search

In [ ]:
import faiss
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors

# Euclidean Distance - kNN

In [ ]:
# Eucledian distance
index_bertforpatents = faiss.IndexFlatL2(1024)   # build the index
index_climatebert = faiss.IndexFlatL2(768)   # build the index
index_patentsberta = faiss.IndexFlatL2(768)   # build the index

# Add df_keywords_claims column 'keyword_yake_bertforpatents_embedding' to index
index_bertforpatents.add(np.array(df_keywords_claims['keyword_yake_bertforpatents_embedding'].tolist()))
index_climatebert.add(np.array(df_keywords_claims['keyword_yake_climatebert_embedding'].tolist()))
index_patentsberta.add(np.array(df_keywords_claims['keyword_yake_patentsberta_embedding'].tolist()))

query_vector_bertforpatents = np.array(df_search_test_titles['keyword_bertforpatents_embedding'][0]).reshape(1, -1)
query_vector_climatebert = np.array(df_search_test_titles['keyword_climatebert_embedding'][0]).reshape(1, -1)
query_vector_patentsberta = np.array(df_search_test_titles['keyword_patentsberta_embedding'][0]).reshape(1, -1)

In [ ]:
k = 25
D_bertforpatents, I_bertforpatents = index_bertforpatents.search(query_vector_bertforpatents, k)
D_climatebert, I_climatebert = index_climatebert.search(query_vector_climatebert, k)
D_patentsberta, I_patentsberta = index_patentsberta.search(query_vector_patentsberta, k)

In [ ]:
# Print row of df_keywords_claims that match the index
df_keywords_claims.iloc[I_patentsberta[0]]

# Cosine Similarity - kNN

In [ ]:
# Cosine similarity
index_bertforpatents = faiss.index_factory(1024, "Flat", faiss.METRIC_INNER_PRODUCT)
index_climatebert = faiss.index_factory(768, "Flat", faiss.METRIC_INNER_PRODUCT)
index_patentsberta = faiss.index_factory(768, "Flat", faiss.METRIC_INNER_PRODUCT)

faiss.normalize_L2(np.array(df_keywords_claims['keyword_yake_bertforpatents_embedding'].tolist()))
faiss.normalize_L2(np.array(df_keywords_claims['keyword_yake_climatebert_embedding'].tolist()))
faiss.normalize_L2(np.array(df_keywords_claims['keyword_yake_patentsberta_embedding'].tolist()))

index_bertforpatents.add(np.array(df_keywords_claims['keyword_yake_bertforpatents_embedding'].tolist()))
index_climatebert.add(np.array(df_keywords_claims['keyword_yake_climatebert_embedding'].tolist()))
index_patentsberta.add(np.array(df_keywords_claims['keyword_yake_patentsberta_embedding'].tolist()))

query_vector_bertforpatents = np.array(df_search_test_titles['keyword_yake_bertforpatents_embedding'][0]).reshape(1, -1)
query_vector_climatebert = np.array(df_search_test_titles['keyword_yake_climatebert_embedding'][0]).reshape(1, -1)
query_vector_patentsberta = np.array(df_search_test_titles['keyword_yake_patentsberta_embedding'][0]).reshape(1, -1)

faiss.normalize_L2(query_vector_bertforpatents)
faiss.normalize_L2(query_vector_climatebert)
faiss.normalize_L2(query_vector_patentsberta)

In [ ]:
k = 6
D_bertforpatents, I_bertforpatents = index_bertforpatents.search(query_vector_bertforpatents, k)
D_climatebert, I_climatebert = index_climatebert.search(query_vector_climatebert, k)
D_patentsberta, I_patentsberta = index_patentsberta.search(query_vector_patentsberta, k)

In [ ]:
# Print row of df_keywords_claims that match the index
df_keywords_claims.iloc[I_bertforpatents[0]]

# Cosine Similarity - Margin

In [ ]:
# Convert lists of embeddings to a 2D array
bertforpatents_embeddings = np.vstack(df_keywords_claims['keyword_yake_bertforpatents_embedding'].apply(np.array))
climatebert_embeddings = np.vstack(df_keywords_claims['keyword_yake_climatebert_embedding'].apply(np.array))
patentsberta_embeddings = np.vstack(df_keywords_claims['keyword_yake_patentsberta_embedding'].apply(np.array))

# Normalize embeddings
bertforpatents_embeddings_normalized = normalize(bertforpatents_embeddings)
climatebert_embeddings_normalized = normalize(climatebert_embeddings)
patentsberta_embeddings_normalized = normalize(patentsberta_embeddings)

# Initialize NearestNeighbors with radius
radius = 0.25
nn_bertforpatents = NearestNeighbors(radius=radius, metric='cosine')
nn_climatebert = NearestNeighbors(radius=radius, metric='cosine')
nn_patentsberta = NearestNeighbors(radius=radius, metric='cosine')

# Fit the models with normalized embeddings
nn_bertforpatents.fit(bertforpatents_embeddings_normalized)
nn_climatebert.fit(climatebert_embeddings_normalized)
nn_patentsberta.fit(patentsberta_embeddings_normalized)

In [ ]:
df_keywords_titles.head()

In [ ]:
def get_keywords_by_indices(indices, df_source, k_neighbors=k_neighbors):
    return df_source.iloc[indices[:k_neighbors]]['keyword_yake_lemma'].tolist() if len(indices) else []

def process_embeddings(row, nn_model, df_keywords_claims, k_neighbors=100):
    query_vector = normalize(np.array([row]).reshape(1, -1))
    distances, indices = nn_model.radius_neighbors(query_vector)
    neighbors = indices[0][:k_neighbors] if indices[0].size else np.array([])
    keywords = get_keywords_by_indices(neighbors, df_keywords_claims, k_neighbors)
    return neighbors.tolist(), keywords

# Define the embeddings to process
embeddings = ['keyword_yake_bertforpatents_embedding', 'keyword_yake_climatebert_embedding', 'keyword_yake_patentsberta_embedding']
nn_models = [nn_bertforpatents, nn_climatebert, nn_patentsberta]
k_neighbors = 100  # Maximum number of neighbors to return

# Process each embedding
for embedding, nn_model in zip(embeddings, nn_models):
    neighbors_col = f'neighbors_{embedding}'
    keywords_col = f'keywords_{embedding}'

    results = df_keywords_titles[embedding].progress_apply(lambda x: process_embeddings(x, nn_model, df_keywords_claims, k_neighbors))
    df_keywords_titles[neighbors_col] = results.progress_apply(lambda x: x[0])
    df_keywords_titles[keywords_col] = results.progress_apply(lambda x: x[1])

In [ ]:
df_keywords_titles.to_json(f"/mnt/hdd01/patentsview/Similarity Search - CPC Classification and Claims/Similarity Search/df_keyword_titles_cosine_similarity_radius_{str(radius).replace('.','')}_neighbors_{str(k_neighbors)}_noun_chunks.json", orient='records')